# Pair Trading BCHUSDT-BTCUSDT

This notebook uses a cryptocurrency trading strategy for BCHUSDT-BTCUSDT and search for the perido window with the heigth return during the timeframe.

Data comes from Biancne and is hourly data between 2019-11-28 to 2020-10-19.

References:

https://www.backtrader.com/docu/pandas-datafeed/pandas-datafeed/

https://github.com/mementum/backtrader/blob/master/contrib/samples/pair-trading/pair-trading.py

https://www.backtrader.com/blog/posts/2015-08-12-observers-and-statistics/observers-and-statistics/



In [8]:
import pandas as pd

In [9]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

In [4]:
%matplotlib inline
# import packages
from binance.client import Client
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sklearn as skl
import pytz
import statsmodels.api as sm
#import zipline
import pathlib

# get api keys for binance 
import os
API_PUBLIC = '<>'
API_SECRET = '<>'
CSV_PATH_PREFIX = os.environ.get("PATH_TO_PROJECT")

crypto_pairs = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BCHUSDT', 'BNBUSDT']

# iteratively loop through cryptocurrencies in crypto_pairs and download them into dataframes
# set granularity of time series
timeperiod = '1h' # hourly
client = Client(API_PUBLIC, API_SECRET)
timestamp = []
for pair in crypto_pairs:
    # determine first available price point
    timestamp.append(client._get_earliest_valid_timestamp(pair, timeperiod)) 
# assign first available price point to be be the latest in time
timestamp = max(timestamp)

startdate = timestamp #"1 Jan, 2019"
df = {} # create a dictionary that will eventually contain currency dataframes
for pair in crypto_pairs:
    klines = client.get_historical_klines(pair, timeperiod, startdate)
    
    # delete unwanted data - just keep date, open, high, low, close
    for line in klines:
        del line[6:]
        
    # cast into dataframe and index with date
    df[pair] = pd.DataFrame(klines, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df[pair]['date'] = pd.to_datetime(df[pair]['date'],unit='ms')
    df[pair].set_index('date', inplace=True)
    
    # previous testing showed that the columns of df are objects. convert to float
    df[pair] = df[pair].astype(float)
    
for key in df:
    df[key].to_csv('hourly/'+key + '.csv')
    print(key)

# import data
df_hourly = {}
for key in os.listdir('hourly'):
    print(key[:-4]) # the weird slicing is to drop the .csv extension
    df_hourly[key[:-4]] = pd.read_csv('hourly/' + key)
    df_hourly[key[:-4]]['date'] = pd.to_datetime(df_hourly[key[:-4]]['date'])# ,unit='ms')
    df_hourly[key[:-4]].set_index('date', inplace=True)

BCHUSDT
BNBUSDT
ETHUSDT
BTCUSDT
XRPUSDT


In [5]:
df = df_hourly

In [30]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import os

import argparse
import datetime

# The above could be sent to an independent module
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

#create datafeed class for pandas dataframes
class PandasData(btfeeds.feed.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
    )


class PairTradingStrategy(bt.Strategy):
    params = dict(
        period=10,
        stake=10,
        qty1=0,
        qty2=0,
        printout=True,
        upper=2.1,
        lower=-2.1,
        up_medium=0.5,
        low_medium=-0.5,
        status=0,
        portfolio_value=10000,
        name1='none',
        name2='none'
    )

    def start(self):
        #record results
        if not os.path.exists('mystats_hour.csv'):
            self.mystats = open('mystats_hour.csv', 'w')
            self.mystats.write('date,pair1,pair2,portfolio,period,status,zscore,pair1_pos,pair2_pos,order_flag\n')
        else:
            self.mystats = open('mystats_hour.csv', 'a')
            
        if not os.path.exists('endstats_hour.csv'):
            self.mystats2 = open('endstats_hour.csv', 'w')
            self.mystats2.write('pair1,pair2,portfolio,period\n')
        else:
            self.mystats2 = open('endstats_hour.csv', 'a')

        
    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.qty1 = self.p.qty1
        self.qty2 = self.p.qty2
        self.upper_limit = self.p.upper
        self.lower_limit = self.p.lower
        self.up_medium = self.p.up_medium
        self.low_medium = self.p.low_medium
        self.status = self.p.status
        self.portfolio_value = self.p.portfolio_value
        self.pair1 = self.p.name1
        self.pair2 = self.p.name2

        # Signals performed with PD.OLS :
        self.transform = btind.OLS_TransformationN(self.data0, self.data1,
                                                   period=self.p.period)
        self.zscore = self.transform.zscore

    def next(self):
        
        

        if self.orderid:
            self.mystats.write(self.data.datetime.datetime(0).strftime('%Y-%m-%dT%H:%M:%S'))
            self.mystats.write(',{}'.format(self.pair1))

            self.mystats.write(',{}'.format(self.pair2))
            self.mystats.write(',{}'.format(self.broker.getvalue()))
            self.mystats.write(',{}'.format(self.p.period))
            self.mystats.write(',{}'.format(self.status))
            self.mystats.write(',{}'.format(self.zscore[0]))
            self.mystats.write(',{}'.format(self.getposition(self.data0).size))
            self.mystats.write(',{}'.format(self.getposition(self.data1).size))
            self.mystats.write(',{}'.format(True))
            self.mystats.write('\n')
            return  # if an order is active, no new orders are allowed

        # Step 2: Check conditions for SHORT & place the order
        # Checking the condition for SHORT
        if (self.zscore[0] > self.upper_limit) and (self.status != 1):

            # Calculating the number of shares for each stock
            value = 0.5 * self.portfolio_value  # Divide the cash equally
            x = int(value / (self.data0.close))  # Find the number of shares for Stock1
            y = int(value / (self.data1.close))  # Find the number of shares for Stock2
            print('x + self.qty1 is', x + self.qty1)
            print('y + self.qty2 is', y + self.qty2)

            # Placing the order
            self.log('SELL CREATE %s, price = %.2f, qty = %d' % (name1, self.data0.close[0], x + self.qty1))
            self.sell(data=self.data0, size=(x + self.qty1))  # Place an order for buying y + qty2 shares
            self.log('BUY CREATE %s, price = %.2f, qty = %d' % (name2, self.data1.close[0], y + self.qty2))
            self.buy(data=self.data1, size=(y + self.qty2))  # Place an order for selling x + qty1 shares

            # Updating the counters with new value
            self.qty1 = x  # The new open position quantity for Stock1 is x shares
            self.qty2 = y  # The new open position quantity for Stock2 is y shares

            self.status = 1  # The current status is "short the spread"

            # Step 3: Check conditions for LONG & place the order
            # Checking the condition for LONG
        elif (self.zscore[0] < self.lower_limit) and (self.status != 2):
            # Calculating the number of shares for each stock
            value = 0.5 * self.portfolio_value  # Divide the cash equally
            x = int(value / (self.data0.close))  # Find the number of shares for Stock1
            y = int(value / (self.data1.close))  # Find the number of shares for Stock2
            print('x + self.qty1 is', x + self.qty1)
            print('y + self.qty2 is', y + self.qty2)

            # Place the order
            self.log('BUY CREATE %s, price = %.2f, qty = %d' % (name1, self.data0.close[0], x + self.qty1))
            self.buy(data=self.data0, size=(x + self.qty1))  # Place an order for buying x + qty1 shares
            self.log('SELL CREATE %s, price = %.2f, qty = %d' % (name2, self.data1.close[0], y + self.qty2))
            self.sell(data=self.data1, size=(y + self.qty2))  # Place an order for selling y + qty2 shares

            # Updating the counters with new value
            self.qty1 = x  # The new open position quantity for Stock1 is x shares
            self.qty2 = y  # The new open position quantity for Stock2 is y shares
            self.status = 2  # The current status is "long the spread"


        # Step 4: Check conditions for No Trade
        # If the z-score is within the two bounds, close all
        
        elif (self.zscore[0] < self.up_medium and self.zscore[0] > self.low_medium and self.status != 0):
            self.log('CLOSE Position %s, price = %.2f, qty = %d' % (name1, self.data0.close[0],self.getposition(self.data0).size))
            self.close(self.data0)
            self.log('CLOSE Position %s, price = %.2f, qty = %d' % (name2, self.data1.close[0],self.getposition(self.data1).size))
            self.close(self.data1)
            self.status = 0
            
        self.mystats.write(self.data.datetime.datetime(0).strftime('%Y-%m-%dT%H:%M:%S'))
        self.mystats.write(',{}'.format(self.pair1))
        
        self.mystats.write(',{}'.format(self.pair2))
        self.mystats.write(',{}'.format(self.broker.getvalue()))
        self.mystats.write(',{}'.format(self.p.period))
        self.mystats.write(',{}'.format(self.status))
        self.mystats.write(',{}'.format(self.zscore[0]))
        self.mystats.write(',{}'.format(self.getposition(self.data0).size))
        self.mystats.write(',{}'.format(self.getposition(self.data1).size))
        self.mystats.write(',{}'.format(False))
        self.mystats.write('\n')
        

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')
        self.mystats2.write('{}'.format(self.pair1))
        
        self.mystats2.write(',{}'.format(self.pair2))
        self.mystats2.write(',{}'.format(self.broker.getvalue()))
        self.mystats2.write(',{}'.format(self.p.period))
        self.mystats2.write('\n')
        self.mystats.close()
        self.mystats2.close()


def runstrategy(name1,name2,period,cash):


    # Create a cerebro
    cerebro = bt.Cerebro()

    # Create the 1st data
    data0 = bt.feeds.PandasData(dataname=df[name1])

    # Add the 1st data to cerebro
    cerebro.adddata(data0)

    # Create the 2nd data
    data1 = bt.feeds.PandasData(dataname=df[name2])

    # Add the 2nd data to cerebro
    cerebro.adddata(data1)

    # Add the strategy
    cerebro.addstrategy(PairTradingStrategy,
                        period=period,
                        stake=10,name1=name1,name2=name2,status=0)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcash(cash)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcommission(commission=0.0)

    # And run it
    cerebro.run()

In [32]:
# search for results for different periods and pairs
# note order of the pair matters
for period in range(96,1008,24):
    for name1 in ['BCHUSDT']:
        for name2 in ['BTCUSDT']:
            if name1==name2:
                None
            else:
                print(period,name1,name2)
                runstrategy(name1,name2,period,10000)

96 BCHUSDT BTCUSDT
x + self.qty1 is 23
y + self.qty2 is 0
2019-12-03T15:00:00, BUY CREATE BCHUSDT, price = 212.20, qty = 23
2019-12-03T15:00:00, SELL CREATE BTCUSDT, price = 7335.43, qty = 0
2019-12-03T16:00:00, BUY COMPLETE, 212.46
2019-12-03T16:00:00, CLOSE Position BCHUSDT, price = 213.10, qty = 23
2019-12-03T16:00:00, CLOSE Position BTCUSDT, price = 7320.47, qty = 0
2019-12-03T17:00:00, SELL COMPLETE, 213.12
x + self.qty1 is 47
y + self.qty2 is 0
2019-12-04T02:00:00, SELL CREATE BCHUSDT, price = 205.99, qty = 47
2019-12-04T02:00:00, BUY CREATE BTCUSDT, price = 7148.36, qty = 0
2019-12-04T03:00:00, SELL COMPLETE, 205.97
2019-12-04T04:00:00, CLOSE Position BCHUSDT, price = 205.17, qty = -47
2019-12-04T04:00:00, CLOSE Position BTCUSDT, price = 7149.44, qty = 0
2019-12-04T05:00:00, BUY COMPLETE, 205.18
x + self.qty1 is 47
y + self.qty2 is 0
2019-12-04T18:00:00, BUY CREATE BCHUSDT, price = 212.70, qty = 47
2019-12-04T18:00:00, SELL CREATE BTCUSDT, price = 7471.21, qty = 0
2019-12-04T19:

2020-01-17T21:00:00, CLOSE Position BCHUSDT, price = 370.00, qty = -29
2020-01-17T21:00:00, CLOSE Position BTCUSDT, price = 8922.34, qty = 0
2020-01-17T22:00:00, BUY COMPLETE, 369.91
x + self.qty1 is 26
y + self.qty2 is 0
2020-01-18T02:00:00, SELL CREATE BCHUSDT, price = 388.94, qty = 26
2020-01-18T02:00:00, BUY CREATE BTCUSDT, price = 8921.21, qty = 0
2020-01-18T03:00:00, SELL COMPLETE, 388.94
2020-01-18T05:00:00, CLOSE Position BCHUSDT, price = 356.87, qty = -26
2020-01-18T05:00:00, CLOSE Position BTCUSDT, price = 8845.19, qty = 0
2020-01-18T06:00:00, BUY COMPLETE, 356.99
x + self.qty1 is 26
y + self.qty2 is 0
2020-01-18T15:00:00, BUY CREATE BCHUSDT, price = 342.36, qty = 26
2020-01-18T15:00:00, SELL CREATE BTCUSDT, price = 8903.26, qty = 0
2020-01-18T16:00:00, BUY COMPLETE, 342.80
2020-01-19T01:00:00, CLOSE Position BCHUSDT, price = 347.32, qty = 26
2020-01-19T01:00:00, CLOSE Position BTCUSDT, price = 9146.02, qty = 0
2020-01-19T02:00:00, SELL COMPLETE, 347.70
x + self.qty1 is 29
y 

2020-03-05T10:00:00, CLOSE Position BCHUSDT, price = 340.34, qty = -29
2020-03-05T10:00:00, CLOSE Position BTCUSDT, price = 9086.41, qty = 0
2020-03-05T11:00:00, BUY COMPLETE, 340.26
x + self.qty1 is 28
y + self.qty2 is 0
2020-03-06T05:00:00, SELL CREATE BCHUSDT, price = 349.24, qty = 28
2020-03-06T05:00:00, BUY CREATE BTCUSDT, price = 9107.86, qty = 0
2020-03-06T06:00:00, SELL COMPLETE, 349.24
2020-03-06T08:00:00, CLOSE Position BCHUSDT, price = 348.30, qty = -28
2020-03-06T08:00:00, CLOSE Position BTCUSDT, price = 9114.71, qty = 0
2020-03-06T09:00:00, BUY COMPLETE, 348.22
x + self.qty1 is 32
y + self.qty2 is 0
2020-03-08T23:00:00, SELL CREATE BCHUSDT, price = 272.77, qty = 32
2020-03-08T23:00:00, BUY CREATE BTCUSDT, price = 8033.31, qty = 0
2020-03-09T00:00:00, SELL COMPLETE, 272.90
x + self.qty1 is 36
y + self.qty2 is 0
2020-03-09T03:00:00, BUY CREATE BCHUSDT, price = 266.50, qty = 36
2020-03-09T03:00:00, SELL CREATE BTCUSDT, price = 8063.52, qty = 0
2020-03-09T04:00:00, BUY COMPLET

x + self.qty1 is 41
y + self.qty2 is 0
2020-04-26T13:00:00, SELL CREATE BCHUSDT, price = 247.75, qty = 41
2020-04-26T13:00:00, BUY CREATE BTCUSDT, price = 7680.11, qty = 0
2020-04-26T14:00:00, SELL COMPLETE, 247.58
x + self.qty1 is 40
y + self.qty2 is 0
2020-04-27T00:00:00, BUY CREATE BCHUSDT, price = 245.71, qty = 40
2020-04-27T00:00:00, SELL CREATE BTCUSDT, price = 7741.66, qty = 0
2020-04-27T01:00:00, BUY COMPLETE, 245.78
2020-04-27T02:00:00, CLOSE Position BCHUSDT, price = 246.23, qty = -1
2020-04-27T02:00:00, CLOSE Position BTCUSDT, price = 7734.98, qty = 0
2020-04-27T03:00:00, BUY COMPLETE, 246.24
x + self.qty1 is 40
y + self.qty2 is 0
2020-04-27T05:00:00, BUY CREATE BCHUSDT, price = 242.57, qty = 40
2020-04-27T05:00:00, SELL CREATE BTCUSDT, price = 7698.00, qty = 0
2020-04-27T06:00:00, BUY COMPLETE, 242.56
2020-04-27T19:00:00, CLOSE Position BCHUSDT, price = 238.60, qty = 40
2020-04-27T19:00:00, CLOSE Position BTCUSDT, price = 7698.32, qty = 0
2020-04-27T20:00:00, SELL COMPLETE,

x + self.qty1 is 41
y + self.qty2 is 0
2020-06-22T02:00:00, SELL CREATE BCHUSDT, price = 235.32, qty = 41
2020-06-22T02:00:00, BUY CREATE BTCUSDT, price = 9386.63, qty = 0
2020-06-22T03:00:00, SELL COMPLETE, 235.32
2020-06-22T04:00:00, CLOSE Position BCHUSDT, price = 235.60, qty = -41
2020-06-22T04:00:00, CLOSE Position BTCUSDT, price = 9411.08, qty = 0
2020-06-22T05:00:00, BUY COMPLETE, 235.62
x + self.qty1 is 41
y + self.qty2 is 0
2020-06-22T18:00:00, SELL CREATE BCHUSDT, price = 241.92, qty = 41
2020-06-22T18:00:00, BUY CREATE BTCUSDT, price = 9581.62, qty = 0
2020-06-22T19:00:00, SELL COMPLETE, 241.92
x + self.qty1 is 40
y + self.qty2 is 0
2020-06-22T22:00:00, BUY CREATE BCHUSDT, price = 240.86, qty = 40
2020-06-22T22:00:00, SELL CREATE BTCUSDT, price = 9678.85, qty = 0
2020-06-22T23:00:00, BUY COMPLETE, 240.87
2020-06-23T03:00:00, CLOSE Position BCHUSDT, price = 240.75, qty = -1
2020-06-23T03:00:00, CLOSE Position BTCUSDT, price = 9658.22, qty = 0
2020-06-23T04:00:00, BUY COMPLETE

...

x + self.qty1 is 20
y + self.qty2 is 0
2020-01-08T22:00:00, SELL CREATE BCHUSDT, price = 240.20, qty = 20
2020-01-08T22:00:00, BUY CREATE BTCUSDT, price = 8076.13, qty = 0
2020-01-08T23:00:00, SELL COMPLETE, 240.23
2020-01-09T03:00:00, CLOSE Position BCHUSDT, price = 238.26, qty = -20
2020-01-09T03:00:00, CLOSE Position BTCUSDT, price = 7954.72, qty = 0
2020-01-09T04:00:00, BUY COMPLETE, 238.26
x + self.qty1 is 41
y + self.qty2 is 0
2020-01-09T08:00:00, BUY CREATE BCHUSDT, price = 236.06, qty = 41
2020-01-09T08:00:00, SELL CREATE BTCUSDT, price = 7933.10, qty = 0
2020-01-09T09:00:00, BUY COMPLETE, 236.06
2020-01-09T09:00:00, CLOSE Position BCHUSDT, price = 234.74, qty = 41
2020-01-09T09:00:00, CLOSE Position BTCUSDT, price = 7872.35, qty = 0
2020-01-09T10:00:00, SELL COMPLETE, 234.70
x + self.qty1 is 42
y + self.qty2 is 0
2020-01-09T22:00:00, SELL CREATE BCHUSDT, price = 236.02, qty = 42
2020-01-09T22:00:00, BUY CREATE BTCUSDT, price = 7811.31, qty = 0
2020-01-09T23:00:00, SELL COMPLET

x + self.qty1 is 27
y + self.qty2 is 0
2020-02-26T08:00:00, BUY CREATE BCHUSDT, price = 323.38, qty = 27
2020-02-26T08:00:00, SELL CREATE BTCUSDT, price = 9195.67, qty = 0
2020-02-26T09:00:00, BUY COMPLETE, 323.15
2020-02-26T13:00:00, CLOSE Position BCHUSDT, price = 319.51, qty = 27
2020-02-26T13:00:00, CLOSE Position BTCUSDT, price = 9092.67, qty = 0
2020-02-26T14:00:00, SELL COMPLETE, 319.34
x + self.qty1 is 30
y + self.qty2 is 0
2020-02-26T18:00:00, SELL CREATE BCHUSDT, price = 324.05, qty = 30
2020-02-26T18:00:00, BUY CREATE BTCUSDT, price = 8842.49, qty = 0
2020-02-26T19:00:00, SELL COMPLETE, 324.00
2020-02-26T21:00:00, CLOSE Position BCHUSDT, price = 316.05, qty = -30
2020-02-26T21:00:00, CLOSE Position BTCUSDT, price = 8748.21, qty = 0
2020-02-26T22:00:00, BUY COMPLETE, 316.05
x + self.qty1 is 30
y + self.qty2 is 0
2020-02-27T01:00:00, SELL CREATE BCHUSDT, price = 329.59, qty = 30
2020-02-27T01:00:00, BUY CREATE BTCUSDT, price = 8694.19, qty = 0
2020-02-27T02:00:00, SELL COMPLET

x + self.qty1 is 42
y + self.qty2 is 0
2020-06-27T21:00:00, BUY CREATE BCHUSDT, price = 217.77, qty = 42
2020-06-27T21:00:00, SELL CREATE BTCUSDT, price = 9062.99, qty = 0
2020-06-27T22:00:00, BUY COMPLETE, 217.80
2020-06-28T00:00:00, CLOSE Position BCHUSDT, price = 215.51, qty = 42
2020-06-28T00:00:00, CLOSE Position BTCUSDT, price = 8982.51, qty = 0
2020-06-28T01:00:00, SELL COMPLETE, 215.52
x + self.qty1 is 42
y + self.qty2 is 0
2020-07-06T20:00:00, SELL CREATE BCHUSDT, price = 238.61, qty = 42
2020-07-06T20:00:00, BUY CREATE BTCUSDT, price = 9274.39, qty = 0
2020-07-06T21:00:00, SELL COMPLETE, 238.63
2020-07-06T23:00:00, CLOSE Position BCHUSDT, price = 242.34, qty = -42
2020-07-06T23:00:00, CLOSE Position BTCUSDT, price = 9344.20, qty = 0
2020-07-07T00:00:00, BUY COMPLETE, 242.35
x + self.qty1 is 40
y + self.qty2 is 0
2020-07-09T06:00:00, SELL CREATE BCHUSDT, price = 244.94, qty = 40
2020-07-09T06:00:00, BUY CREATE BTCUSDT, price = 9412.00, qty = 0
2020-07-09T07:00:00, SELL COMPLET

x + self.qty1 is 31
y + self.qty2 is 0
2020-08-16T01:00:00, SELL CREATE BCHUSDT, price = 313.20, qty = 31
2020-08-16T01:00:00, BUY CREATE BTCUSDT, price = 11890.57, qty = 0
2020-08-16T02:00:00, SELL COMPLETE, 313.08
2020-08-16T05:00:00, CLOSE Position BCHUSDT, price = 307.95, qty = 1
2020-08-16T05:00:00, CLOSE Position BTCUSDT, price = 11900.04, qty = 0
2020-08-16T06:00:00, SELL COMPLETE, 307.92
x + self.qty1 is 30
y + self.qty2 is 0
2020-08-17T09:00:00, SELL CREATE BCHUSDT, price = 312.83, qty = 30
2020-08-17T09:00:00, BUY CREATE BTCUSDT, price = 11856.31, qty = 0
2020-08-17T10:00:00, SELL COMPLETE, 312.84
2020-08-17T11:00:00, CLOSE Position BCHUSDT, price = 314.89, qty = -30
2020-08-17T11:00:00, CLOSE Position BTCUSDT, price = 11908.65, qty = 0
2020-08-17T12:00:00, BUY COMPLETE, 314.81
x + self.qty1 is 31
y + self.qty2 is 0
2020-08-21T03:00:00, SELL CREATE BCHUSDT, price = 301.52, qty = 31
2020-08-21T03:00:00, BUY CREATE BTCUSDT, price = 11866.58, qty = 0
2020-08-21T04:00:00, SELL CO

x + self.qty1 is 37
y + self.qty2 is 0
2020-10-16T16:00:00, BUY CREATE BCHUSDT, price = 250.43, qty = 37
2020-10-16T16:00:00, SELL CREATE BTCUSDT, price = 11344.53, qty = 0
2020-10-16T17:00:00, BUY COMPLETE, 250.48
2020-10-16T20:00:00, CLOSE Position BCHUSDT, price = 249.89, qty = 37
2020-10-16T20:00:00, CLOSE Position BTCUSDT, price = 11313.09, qty = 0
2020-10-16T21:00:00, SELL COMPLETE, 249.90
x + self.qty1 is 39
y + self.qty2 is 0
2020-10-17T14:00:00, BUY CREATE BCHUSDT, price = 246.75, qty = 39
2020-10-17T14:00:00, SELL CREATE BTCUSDT, price = 11356.74, qty = 0
2020-10-17T15:00:00, BUY COMPLETE, 246.81
2020-10-17T22:00:00, CLOSE Position BCHUSDT, price = 243.66, qty = 39
2020-10-17T22:00:00, CLOSE Position BTCUSDT, price = 11343.50, qty = 0
2020-10-17T23:00:00, SELL COMPLETE, 243.69
Starting Value - 10000.00
Ending   Value - 17084.27


In [34]:
#final results saved in endstats.csv
results = pd.read_csv('endstats_hour.csv')

In [35]:
#get best result
results[results.portfolio==results.portfolio.max()]

,pair1,pair2,portfolio,period
7,BCHUSDT,BTCUSDT,19455.73,264
